# Variational multi-scale finite element solution of the compressible <b> 
# Navier-Stokes equations - Automatic Differentiation Formulation

## <center>Elisa Magliozzi<center> 

### <center> *November 2017*<center>


## List of Symbols

In [2]:
from KratosMultiphysics import *

from sympy import *
from sympy_fe_utilities import *
import pprint

from params_dict import params
import ConvectiveFlux
import DiffusiveFlux
import SourceTerm
import StabilizationMatrix

dim = params["dim"]  
dimes = dim+2 					        # Dimension of the vector of Unknowns
do_simplifications = False
#dim_to_compute = "Both"                # Spatial dimensions to compute. Options:  "2D","3D","Both"
mode = "c"                              # Output mode to a c++ file

if(dim == 2):
    nnodes = 3
elif(dim == 3):
    nnodes = 4

impose_partion_of_unity = False
N,DN = DefineShapeFunctions(nnodes, dim, impose_partion_of_unity)
  
# Unknown fields definition (Used later for the gauss point interpolation)
U = DefineMatrix('U',nnodes,dimes)	     # Vector of Unknowns ( Density,Velocity[dim],Total Energy )
Un = DefineMatrix('Un',nnodes,dimes)     # Vector of Unknowns one step back
Unn = DefineMatrix('Unn',nnodes,dimes)   # Vector of Unknowns two steps back
r = DefineVector('r',nnodes)             # Sink term

# Test functions defintiion
w = DefineMatrix('w',nnodes,dimes)	     # Variables field test

# External terms definition
f_ext = DefineMatrix('f_ext',nnodes,dim) # Forcing term 

# Definition of other symbols
bdf0 = Symbol('bdf0')                    # Backward differantiation coefficients
bdf1 = Symbol('bdf1')
bdf2 = Symbol('bdf2')


### 1. Compressible Navier-Stokes formulation

A compressible fluid can be described using the equations of conservation of mass, momentum and energy, also known as Navier-Stokes equations. The equations described below are considering a Newtonian, viscous fluid.

1. Conservation of mass equation

    \begin{equation}
    \label{MassConserv}
    \frac{\partial \rho}{\partial t} + \nabla \cdot (\rho \textbf{u})  = 0.
    \end{equation}

2. Conservation of momentum equation

    \begin{equation}
    \label{MomentumCons}
    \frac{\partial (\rho \mathbf{u})}{\partial t} + \nabla \big(\rho \mathbf{u}\big) \mathbf{u} +\nabla \big(p\mathbf{I}-\mathbf{\tau}\big) = \rho \mathbf{f}
    \end{equation}
    
3. Conservation of energy equation

    \begin{equation}
    \label{EnergyCons}
    \frac{\partial}{\partial t} \bigg(\rho \big( e + \frac{1}{2}  \mathbf{u}\cdot\mathbf{u}\big)\bigg) + \nabla \bigg(\rho \mathbf{u} \big(\mathbf{h}+\frac{1}{2}\mathbf{u}\cdot\mathbf{u}\big) - \mathbf{u}\cdot \mathbf{\tau} + \mathbf{q}\bigg)= \rho \mathbf{f}\cdot\mathbf{u}+\rho r
    \end{equation}
    

Where *$\rho$* is the density, *p* is the pressure, * **u** * is the velocity, *$\tau$* is the viscous stress tensor, * **f** * is the body force vector, *e* is the internal energy, *h* is the enthalpy, * **q** * is the heat flux vector and *r* is a heat soure/sink term. 

Now the equation is written in terms of the conservative variable *$\rho$*, * **m** * , and *$e_{tot}$*   *$ = \rho \big( e + \frac{1}{2}  \mathbf{u}\cdot\mathbf{u}\big)$*

1. Conservation of mass equation

    \begin{equation}
    \label{MassConserv}
    \frac{\partial \rho}{\partial t} + \nabla \cdot (\textbf{m})  = 0.
    \end{equation}

2. Conservation of momentum equation

    \begin{equation}
    \label{MomentumCons}
    \frac{\partial (\mathbf{m})}{\partial t} + \nabla \cdot (\mathbf{m}) \frac{\mathbf{m}}{\rho} +\nabla (p\mathbf{I}-\mathbf{\tau}) = \rho \mathbf{f}
    \end{equation}
    
3. Conservation of energy equation

    \begin{equation}
    \label{EnergyCons}
    \frac{\partial e_{tot}}{\partial t} + \nabla \bigg((\rho+e_{tot})\frac{\mathbf{m}}{\rho} -\frac{\mathbf{m}}{\rho}\cdot\mathbf{\tau}+ \mathbf{q}\bigg)= \mathbf{f}\mathbf{m}+\rho r
    \end{equation}
    


It is possible to group the Navier-Stokes equations in a system with the help of the Einstein summation, considering $\mathbf{U} = (\rho, \mathbf{m}, e_{tot})^T$ as the vector of the conservative variables.

\begin{equation}
    \label{System}
    \frac{\partial (\mathbf{U})}{\partial t} + \frac{\partial \mathbf{F}_j (\mathbf{U})}{\partial x_j}+\frac{\partial \mathbf{G}_j (\mathbf{U})}{\partial x_j}-\mathbf{S}(\mathbf{U}) = \mathbf{0}, \quad in \quad \Omega \subset \mathbb{R}^d, t>0,
\end{equation}

\begin{equation}  
\label{Dirichlet}
 \textit{U}(\mathbf{U}_g)=\mathbf{U}_g,\quad \quad \quad \quad \quad \quad \quad \quad on\quad \Gamma_g,t>0,
 \end{equation}
 
 \begin{equation}
     \label{Neumann}
     \mathbf{F}_jn_j=\mathbf{h}, \quad \quad \quad \quad \quad \quad \quad \quad\quad \quad on\quad \Gamma_n, t>0,
 \end{equation}
 
 \begin{equation}
     \label{InitialCond}
     \mathbf{U}=\mathbf{U}_0(\mathbf{x}), \quad \quad \quad \quad \quad \quad \quad \quad\quad \quad in\quad \Omega, t=0
 \end{equation}
 
 The boundary conditions for the Dirichlet and Neumann boundaries are here introduced, together with the initial condition. 

In [9]:
### Construction of the variational equation

Ug = DefineVector('Ug',dim+2)			# Dofs vector
H = DefineMatrix('H',dim+2,dim)			# Gradient of U
f = DefineVector('f',dim)			    # Body force vector
rg = Symbol('rg', positive = True)		# Source/Sink term
V = DefineVector('V',dim+2)			    # Test function
Q = DefineMatrix('Q',dim+2,dim)			# Gradient of V
acc = DefineVector('acc',dimes)         # Derivative of Dofs/Time


The $\textit{(d+2)*d}$ $\textit{convective}$ flux matrix $\mathbf{F}$ is:

\begin{equation}
    \label{Fmat}
    \mathbf{F}_j(\mathbf{U}) = \bigg(m_j,\frac{m_j}{\rho}m_i+p\delta_{ij},(e_{tot}+p)\frac{m_j}{\rho}\bigg)^T \quad \quad 1\leq{i},j\leq{d}
\end{equation}

From this it is possible to derive the $\textit{(d+2)*(d+2)*d}$ Euler Jacobian matrix $\mathbf{A}_j$.

\begin{equation}
    \label{DiffAmat}
    \frac{\partial \mathbf{F}_j(\mathbf{U})}{\partial x_j}=\mathbf{A}_j(\mathbf{U})\frac{\partial \mathbf{U}}{\partial x_j}
\end{equation}
and so


\begin{equation}
    \label{Amatr}
    \mathbf{A}_j(\mathbf{U}) = \frac{\partial\mathbf{F}_j(\mathbf{U})}{\partial \mathbf{U}}
\end{equation}



In [11]:
A = ConvectiveFlux.computeA(Ug,params)
#ConvectiveFlux.printA(A,params)


Compute Convective Matrix 



In [35]:
## How the Convective Flux Matrix has been implemented:

%run 'ConvectiveFlux.ipynb'

The $\textit{(d+2)*d diffusive}$ flux matrix $\mathbf{G}$ is defined as follow.

\begin{equation}
    \label{Gmat}
    \mathbf{G}_j(\mathbf{U}) = \big(0,-\tau_{ji}, -\frac{m_i}{\rho}\tau_{ij}+q_j\big)^T \quad \quad 1\leq{i},j\leq{d}
\end{equation}

The diffusive matrix $\mathbf{K}_{kj}(\mathbf{U})$ is a fourth order tensor of dimension $\textit{(d+2)*(d+2)*d*d}$ and can be derived from the expression below.

\begin{equation}
    \label{K}
    \frac{\partial\mathbf{G}_j(\mathbf{U})}{\partial x_j} = -\frac{\partial}{\partial x_k}\bigg(\mathbf{K}_{kj}(\mathbf{U})\frac{\partial \mathbf{U}}{\partial x_j}\bigg)
\end{equation}



In [12]:
K = DiffusiveFlux.computeK(Ug,params)
#DiffusiveFlux.printK(K,params)


Compute Diffusive Matrix



In [37]:
## How the Convective Flux Matrix has been implemented:

%run 'DiffusiveFlux.ipynb'

The $\textit{source}$ term vector $\mathbf{S(\mathbf{U})}$ is written as a product of a $\textit{(d+2)*(d+2)}$ reactive matrix $\mathbf{S}$ and the vector of unknowns.

\begin{equation}
    \label{S(U)}
    \mathbf{S}(\mathbf{U}) = (0,\rho\mathbf{f},\mathbf{f}\cdot\mathbf{m}+\rho r)^T
\end{equation}

\begin{equation}
    \label{S}
    \mathbf{S}=
    \begin{pmatrix} 0 & 0 & 0\\ \mathbf{f} & 0 & 0 \\ r &\mathbf{f}^T &0 \end{pmatrix}
\end{equation}

In [14]:
S = SourceTerm.computeS(f,rg,params)
#SourceTerm.printS(S,params)


Compute Source Matrix 



The problem can be rewritten using  the nonlinear operator $\mathbf{\textit{L}}(\mathbf{U};\mathbf{U})$ as below.


\begin{equation}
    \label{system2}
    \frac{\partial \mathbf{U}}{\partial t} + \mathbf{\textit{L}}(\mathbf{U} ; \mathbf{U}) = \mathbf{0}, \quad in \quad \Omega \subset \mathbb{R}^d, t>0,
\end{equation}

\begin{equation}  
\label{Dirichlet}
 \textit{U}(\mathbf{U}_g)=\mathbf{U}_g,\quad \quad \quad \quad \quad \quad \quad \quad on\quad \Gamma_g,t>0,
 \end{equation}
 
 \begin{equation}
     \label{Neumann}
     \mathbf{F}_jn_j=\mathbf{h}, \quad \quad \quad \quad \quad \quad \quad \quad\quad \quad on\quad \Gamma_n, t>0,
 \end{equation}
 
 \begin{equation}
     \label{InitialCond}
     \mathbf{U}=\mathbf{U}_0(\mathbf{x}), \quad \quad \quad \quad \quad \quad \quad \quad\quad \quad in\quad \Omega, t=0
 \end{equation}
Where $\mathbf{\textit{L}}$ is define as here.

\begin{equation}
    \label{L}
    \mathbf{\textit{L}}(\mathbf{U};\mathbf{U}) = \mathbf{A}_j\frac{\partial \mathbf{U}}{\partial x_j}- \frac{\partial}{\partial x_k}\bigg(\mathbf{K}_{kj}(\mathbf{U})\frac{\partial \mathbf{U}}{\partial x_j}\bigg) -\mathbf{S} \mathbf{U}
\end{equation}


In [16]:
## Nonlinear operator definition
L = DefineVector('L',dim+2)		       # Nonlinear operator
res = DefineVector('res',dim+2)		   # Residual definition

l1 = Matrix(zeros(dim+2,1))		       # Convective Matrix*Gradient of U
tmp = []
for j in range(0,dim):
    tmp = A[j]*H[:,j]
    l1 +=tmp

l2 = Matrix(zeros(dim+2,1))		       # Diffusive term

for s in range(0,dim+2):
    for k in range(0,dim):
        kinter = K[k]			       # Intermediate matrix
        for j in range(0,dim):
            ksmall = kinter[j]		   # Intermediate 2 matrix
            for l in range(0,dim+2):
                for m in range(0,dim+2):
                    for n in range(0,dim+2):
                        l2[s] += diff(ksmall[l,m],Ug[n])*H[n,k]*H[s,j]
                        #print("l",l,"m",m,"n",n,":\n",l2[s],"\n\n\n\n\n\n"

l3 = S*Ug				               # Source term
L = l1-l2-l3

### 2. Approximation of the subscales 

The finite element space is subdivided into a coarse and a fine subgrid space.<br>
 <center>$\mathbf{V} = \mathbf{V}_h+\widetilde{\mathbf{V}}, \quad \mathbf{V}_h \subset \textit{W}_h$</center>                    
In the same way, the $\mathbf{U}$ vector is decomposed in: <br>
<center>$ \mathbf{U} = \mathbf{U}_h + \widetilde{\mathbf{U}}, \quad \mathbf{U} \in \textit{W}$<center><br>

In order to understand the final formulation of the variational problem we need to define the nonlinear adjoint operator $\mathbf{\textit{L}^{*}}(\mathbf{U};\mathbf{V}_h)$ and the finite element residual $\mathbf{R}(\mathbf{U};\mathbf{U}_h)$, together with the stabilization matrix $\mathbf{\tau}$.
    
The nonlinear adjoint operator $\mathbf{\textit{L}^{*}}$ is here applied to the test functions vector $\mathbf{V}_h$.

\begin{equation}
    \label{Nonlinoper}
    \mathbf{\textit{L}^{*}}(\mathbf{U};\mathbf{V}_h) =-\frac{\partial}{\partial x_j}\bigg(\mathbf{A}_j^{T}(\mathbf{U})\mathbf{V}_h\bigg) - \frac{\partial}{\partial x_j} \bigg(\mathbf{K}_{kj}^{T}(\mathbf{U})\frac{\partial \mathbf{V}_h}{\partial x_k}\bigg) - \mathbf{S}^T \mathbf{V}_h
\end{equation}

In [17]:
## Nonlinear adjoint operator definition
L_adj = DefineVector('L_adj',dim+2)	   # Nonlinear adjoint operator

m1 = Matrix(zeros(dim+2,1))		       # Convective term
for s in range(0,dim+2):
    for j in range(0,dim):
        A_T = A[j].transpose()
        for l in range(0,dim+2):
            for m in range(0,dim+2):
                m1[s] -= A_T[l,m]*Q[s,j]
                for n in range(0,dim+2):
                    m1[s] -= diff(A_T[l,m],Ug[n])*H[n,j]*V[s]
                  
m2 = Matrix(zeros(dim+2,1))		       # Diffusive term

for s in range(0,dim+2):
    for k in range(0,dim):
        kinter = K[k]
        for j in range(0,dim):
            ksmall = kinter[j].transpose()
            for l in range(0,dim+2):
                for m in range(0,dim+2):
                    for n in range(0,dim+2):
                        m2[s] -= diff(ksmall[l,m],Ug[n])*H[n,j]*Q[s,k]

m3 = -S.transpose()*V			        # Source term
L_adj = m1+m2+m3

The finite element residual is:
\begin{equation}
    \label{R}
    \mathbf{R}(\mathbf{U};\mathbf{U_h}) = -\frac{\partial \mathbf{U}_h}{\partial t}-\mathbf{\textit{L}}(\mathbf{U};\mathbf{U}_h)
\end{equation}

In [18]:
## Redisual definition
res = -acc - L		

The stabilization matrix $\mathbf{\tau}$ is a $\textit{(d+2)*(d+2)}$ diagonal matrix defined like this:<br>
<br>
\begin{equation}
    \label{tau}
    \boldsymbol{\tau}^{-1} = diag \big(\tau^{-1}_1,\tau^{-1}_2\mathbf{I}_d,\tau^{-1}_3 \big)  \\
    \tau^{-1}_1 = \textit{c}_2 \frac{\lvert {\frac{\mathbf{m}} {\rho}}\rvert +c}{\textit{h}} \\
    \tau^{-1}_2 = \textit{c}_1 \frac{\nu}{\textit{h}^2} +\textit{c}_2 \frac{\lvert {\frac{\mathbf{m}} {\rho}}\rvert +c}{\textit{h}} \\
    \tau^{-1}_3 = \textit{c}_1 \frac{\lambda}{\rho \textit{c}_p \textit{h}^2} +\textit{c}_2 \frac{\lvert {\frac{\mathbf{m}} {\rho}}\rvert +c}{\textit{h}} \\
\end{equation}

Where $\textit{c}$ is the speed of sound, while $\textit{c}_1$ and $\textit{c}_2$ are algorithm constants (here $\textit{c}_1$ = 4 and $\textit{c}_2$ = 2


In [19]:
Tau = StabilizationMatrix.computeTau(Ug,params)
#StabilizationMatrix.printTau(Tau,params)


Compute Stabilization Matrix



Finally the problem can be described using the equation below.

\begin{equation}
    \label{final}
    \bigg(\mathbf{V}_h,\frac{\partial \mathbf{U}_h}{\partial t}\bigg) + \bigg(\mathbf{V}_h,\mathbf{A}_j(\mathbf{U}_h)\frac{\partial \mathbf{U}_h}{\partial x_j}\bigg) + \bigg(\mathbf{V}_h,\mathbf{K}_{kj}(\mathbf{U}_h)\frac{\partial \mathbf{U}_h}{\partial x_j}\bigg) - \bigg(\mathbf{V}_h,\mathbf{S} \mathbf{U}_h\bigg) + \sum_{K} \bigg\langle \mathbf{\textit{L}^{*}}(\mathbf{U}_h;\mathbf{V}_h),
    \boldsymbol{\tau} (\mathbf{U}_h) \mathbf{R}(\mathbf{U}_h) \bigg\rangle_K = 0 \quad \forall \mathbf{V}_h \in  \textit{W}
\end{equation}

In [20]:
## Variational Formulation - Final equation

n1 = V.transpose()*acc		            # Mass term - FE scale

for i in range(0,dim):
    tmp += A[i]*H[:,i]
n2 = V.transpose()*tmp			       # Convective term - FE scale

tmp = Matrix(zeros(dim+2,1))
n3 = Matrix(zeros(1,1))			       # Diffusive term - FE scale
for k in range(0,dim):
    kinter= K[k]
    for j in range(0,dim):
        tmp += kinter[j]*H[:,j] 
    n3 += Q[:,k].transpose()*tmp

n4 = -V.transpose()*(S*Ug)		       # Source term - FE scale

n5 = L_adj.transpose()*(Tau*res)	   # VMS_adjoint - Subscales

rv = n1+n2+n3+n4+n5 			       # VARIATIONAL FORMULATION - FINAL EQUATION

In [ ]:
### Substitution of the discretized values at the gauss points

## Data interpolation at the gauss points
U_gauss = U.transpose()*N
w_gauss = w.transpose()*N
f_gauss = f_ext.transpose()*N
acc_gauss = (bdf0*U+bdf1*Un+bdf2*Unn).transpose()*N
r_gauss = (r.transpose()*N)[0]      

## Gradients computation
grad_U = DfjDxi(DN,U).transpose()
grad_w = DfjDxi(DN,w).transpose()

SubstituteMatrixValue(rv, Ug, U_gauss)
SubstituteMatrixValue(rv, acc, acc_gauss)
SubstituteMatrixValue(rv, H, grad_U)
SubstituteMatrixValue(rv, V, w_gauss)
SubstituteMatrixValue(rv, Q, grad_w)
SubstituteMatrixValue(rv, f, f_gauss)
SubstituteScalarValue(rv, rg, r_gauss)

In [3]:
rhs = Compute_RHS(rv.copy(), w, do_simplifications)
rhs_out = OutputVector_CollectingFactors(rhs, "rhs", "c")

lhs = Compute_LHS(rhs, w, U, do_simplifications) # Compute the LHS
lhs_out = OutputMatrix_CollectingFactors(lhs, "lhs", mode)

NameError: name 'rv' is not defined

In [ ]:
'''
# WRITING TEMPLATE FILE
templatefile = open("compressible_navier_stokes_cpp_template.cpp")
outstring=templatefile.read()

outstring = outstring.replace("//substitute_lhs_2D", "HO LETTO")

out = open("compr_navier_stokes.cpp",'w')
out.write(outstring)
out.close()


if(dim == 2):
        outstring = outstring.replace("//substitute_lhs_2D", lhs_out)
        outstring = outstring.replace("//substitute_rhs_2D", rhs_out)
elif(dim == 3):
        outstring = outstring.replace("//substitute_lhs_3D", lhs_out)
        outstring = outstring.replace("//substitute_rhs_3D", rhs_out)

## Write the modified template
out = open("compressible_navier_stokes.cpp",'w')
out.write(outstring)
out.close()
'''